Creacion de dataset, haciendo scrapping de distintos productos de amazon para extraer las reseñas junto con su etiqueta(numero de estrellas)

In [ ]:
import requests
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd

# Lista de productos a buscar
productos = ["Televisión"]

# URL base de Amazon México
url = "https://www.amazon.com.mx/s"

# Encabezados personalizados
custom_headers = {
    'user-agent': generate_user_agent(),
    'accept-language': 'en-GB,en;q=0.9',
}

# Lista para almacenar los enlaces de productos
product_links_list = []
num_products = 1  # Numero de enlaces a etxraer reseñas por cada tipo de producto
current_url = url

# Extraer enlaces de productos
def get_product_links(product_name):
    global current_url
    product_links_list = []
    params = {'k': product_name}
    
    while len(product_links_list) < num_products:
        response = requests.get(current_url, params=params, headers=custom_headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            product_links = soup.find_all("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")
            
            for product in product_links:
                if len(product_links_list) >= num_products:
                    break
                link = "https://www.amazon.com.mx" + product['href']
                product_links_list.append(link)

            next_page = soup.find("a", class_="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator")
            if next_page and 'href' in next_page.attrs:
                current_url = "https://www.amazon.com.mx" + next_page['href']
            else:
                break
        else:
            print("No se pudo acceder a la página. Código de estado:", response.status_code)
            break

    return product_links_list

# Configuración para Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument(f"user-agent={generate_user_agent()}")

# Funcion para obtener reseñas y calificaciones de Amazon
def get_reviews_amazon(link):
    driver = webdriver.Chrome(options=chrome_options)
    reviews_data = []
    
    try:
        driver.get(link)
        while True:
            reviews = driver.find_elements(By.CLASS_NAME, "review-text-content")
            ratings = driver.find_elements(By.CLASS_NAME, "a-icon-alt")
            
            for review, rating in zip(reviews, ratings):
                try:
                    stars = float(rating.get_attribute('textContent').split(' ')[0])
                    reviews_data.append((review.text, stars))
                except ValueError:
                    continue  # En caso de no poder extraer una calificación válida

            try:
                next_button = driver.find_element(By.XPATH, "//li[@class='a-last']/a")
                if next_button.is_enabled():
                    next_button.click()
                    time.sleep(2)
                else:
                    break
            except (NoSuchElementException, ElementClickInterceptedException):
                break
    finally:
        driver.quit()

    return reviews_data

# Lista para almacenar todas las reseñas con sus calificaciones
all_reviews = []

# Iterar sobre cada tipo de producto
for product in productos:
    print(f"Extrayendo enlaces de productos para: {product}")
    product_links = get_product_links(product)
    
    for link in product_links:
        print(f"Extrayendo reseñas del producto: {link}")
        reviews = get_reviews_amazon(link)
        
        # Guardar las reseñas extraidas
        for review, rating in reviews:
            all_reviews.append({"Producto": product, "Reseña": review, "Calificación": rating})

# DataFrame con las reseñas y calificaciones
df = pd.DataFrame(all_reviews)

# Guardar el DataFrame en un archivo Excel
df.to_excel("reseñas_amazon.xlsx", index=False)

print(f"Se han extraído {len(df)} reseñas y se han guardado en el archivo 'reseñas_amazon.xlsx'.")


Extrayendo enlaces de productos para: Televisión
Extrayendo reseñas del producto: https://www.amazon.com.mx/TCL-Pantalla-32S230A-Android-Compatible/dp/B0C7F5RCYL/ref=sr_1_1?dib=eyJ2IjoiMSJ9.i_jnIt-EDPGeqmxCO_RazIf9jKouIx0QQAk_iZdLkjPCKhucaxazmXhgwHo5axPCQzfCnJAiF3G2D4u1uCycyEZM38G-xpVCLAuYIu5XgV0cgNqIPyI9T3rJqVYmDVxpx1nZI7aJW0Qjda7YlD0sHcA5OqOXTMpaLa0w_-2ntnSA2RUR4wzXHDrugoab3fwKq9Kq20bfmq6OEQlsR5jfQEAZO5uDCqkIuZB9gFa0W6m8nB8TqCsXGhXjL_TiE8lVc_lIfZhXziJUECjLiClG831Q3F_S8k9SU9jdb1Mb75I.EEFD8Hc7JxYaCAICLhup3XCNOfYM6ud66Av9FF243cA&dib_tag=se&keywords=Televisi%C3%B3n&qid=1731005316&sr=8-1&ufe=app_do%3Aamzn1.fos.709e624a-cf89-43d6-ada3-7117189c8ad8
Se han extraído 4 reseñas y se han guardado en el archivo 'reseñas_amazon.xlsx'.


# Lista de productos para buscar para un dataset mas variado
productos = [
    "Televisión", "Laptop", "Celular", "Estufa", "Refrigerador",
    "Microondas", "Lavadora", "Secadora", "Auriculares", "Reloj",
    "Monitor", "Impresora", "Cámara", "Tablet", "Proyector",
    "Altavoz", "Router", "Consola", "Teclado", "Mouse",
    "Batería", "Cargador", "Licuadora", "Aspiradora", "Parrilla",
    "Horno", "Cafetera", "Ventilador", "Aire acondicionado", "Plancha",
    "Tostadora", "Deshumidificador", "Termo", "Humidificador", "Lampara",
    "Radiador", "Purificador", "Extractor", "Estéreo", "Secador",
    "Enchufe", "Cable", "Bocina", "Disco duro", "Memoria",
    "Linterna", "Taladro", "Escoba", "Pala", "Escritorio",
    "Silla", "Mesa", "Colchón", "Cojín", "Edredón",
    "Almohada", "Cortina", "Espejo", "Cuadro", "Vela",
    "Jarrón", "Plato", "Taza", "Vaso", "Cubiertos",
    "Sartén", "Olla", "Cuchillo", "Rallador", "Tabla",
    "Colador", "Batidora", "Freidora", "Exprimidor", "Molde",
    "Cepillo", "Peine", "Toalla", "Bolsas", "Rastrillo",
    "Esponja", "Shampoo", "Jabón", "Perfume", "Desodorante",
    "Bicicleta", "Scooter", "Patineta", "Pelota", "Pesas",
    "Colchoneta", "Botella", "Mochila", "Maleta", "Paraguas",
    "Guantes", "Gorra", "Sombrero", "Calcetines", "Reloj"
]